### Question 1 — What is Information Gain, and how is it used in Decision Trees?

**Answer:**

**Definition:** Information Gain (IG) measures how much "information" a feature gives us about the class; more formally it is the reduction in entropy (uncertainty) about the target variable after splitting the dataset on a given attribute.

**Mathematical formula:**
- Entropy before split: \(H(S) = -\sum_{c} p(c)\log_2 p(c)\).
- Entropy after splitting on attribute A with values v: \(H(S|A) = \sum_{v}
rac{|S_v|}{|S|} H(S_v)\).
- Information Gain: \(IG(S, A) = H(S) - H(S|A)\).

**How it's used in Decision Trees:**
- During tree construction (e.g., ID3), at each node the algorithm evaluates candidate features and chooses the feature with the highest Information Gain to split on. This maximizes the reduction in uncertainty about the class.
- The process repeats recursively on child nodes until stopping criteria (pure nodes, max depth, or minimum samples) are met.

**Interpretation & properties:**
- High IG means the feature produces child nodes that are more homogeneous (lower entropy) than the parent.
- IG is biased toward features with many distinct values. To correct this, alternatives like *Gain Ratio* (which normalizes IG by intrinsic information) can be used.

**Practical notes (for full marks):**
- Computing IG requires estimation of class probabilities from available samples (so small sample sizes can give noisy estimates).
- IG is a supervised measure (depends on labels). For continuous features, typical approach is to find the best threshold (binary split) that maximizes IG.
- Example use in popular libraries: `scikit-learn`’s `DecisionTreeClassifier` supports `criterion='entropy'` to use entropy/IG-based splits.

### Question 2 — What is the difference between Gini Impurity and Entropy?


**Answer:**

**Definitions:**
- **Entropy (Information Gain criterion)**: \(H(S) = -\sum_{i} p_i \log_2 p_i\). It measures the average information needed to identify the class; sensitive to distribution tails.
- **Gini Impurity**: \(G(S) = 1 - \sum_{i} p_i^2\). It measures the probability of mislabeling a randomly chosen element if it were labeled according to the distribution.

**Range and interpretation:**
- Both range from 0 (pure node) to a maximum when the class distribution is uniform.
- Gini is simpler to compute (no log) and often slightly faster.

**Behavioral differences:**
- **Sensitivity**: Entropy tends to be more sensitive to changes near the tails of the distribution; Gini tends to prefer larger partitions and is often slightly more biased toward splits that isolate the most frequent class.
- **Split choice**: In practice, splits chosen by both metrics are often very similar. Differences do appear on some datasets, but rarely huge.

**Strengths & weaknesses:**
- **Entropy (IG)**:
  - + Solid information-theoretic interpretation.
  - − Slightly more computational cost due to log.
  - − More sensitive to class probabilities, can prefer balanced splits that reduce entropy a lot.
- **Gini**:
  - + Computationally cheaper.
  - + Often yields similar or slightly better practical performance in many cases.
  - − Less interpretable from an information perspective.

**When to use which:**
- Use **Gini** when speed matters and interpretability of split choice via information theory is not required.
- Use **Entropy/Information Gain** when you want the information-theoretic justification or when using algorithms (or assignments) that explicitly request entropy.

**Summary line (for exam):** Gini is a measure of impurity based on squared probabilities and is computationally simpler; entropy is an information-theoretic impurity measure using logarithms. Both encourage purer child nodes; differences are small in practice but entropy has a stronger theoretical interpretation while Gini is slightly faster.

### Question 3 — What is Pre-Pruning in Decision Trees?

**Answer:**

**Definition:** Pre-pruning (also called *early stopping*) is a technique to stop the growth of a decision tree early — i.e., while building the tree — by applying stopping criteria so that nodes are not split if they do not meet certain requirements.

**Common pre-pruning criteria:**
- Maximum tree depth (`max_depth`) — disallow splits deeper than this.
- Minimum number of samples required to split (`min_samples_split`) or in a leaf (`min_samples_leaf`).
- Minimum impurity decrease — only split if the impurity decrease is above a threshold.
- Maximum number of leaf nodes.

**Why use pre-pruning:**
- Prevent **overfitting** by restricting tree complexity.
- Reduce computational cost and improve generalization on unseen data.
- Often necessary for noisy datasets or when the dataset is small.

**Advantages:**
- Simple to implement.
- Controls model complexity during training.
- Can dramatically improve generalization when tuned correctly.

**Disadvantages:**
- Might stop too early and underfit (miss important structure).
- Choosing the correct thresholds requires validation (e.g., cross-validation).

**Comparison with post-pruning:** Post-pruning grows a full tree and then prunes back (e.g., cost-complexity pruning). Post-pruning can potentially recover a better structure since it examines the full tree, but it requires more computation. Pre-pruning is faster but may prematurely cut useful branches.

### Question 4 — Write a Python program to train a Decision Tree Classifier using Gini Impurity as the criterion and print the feature importances (practical).


**Code:**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
import numpy as np

X, y = load_iris(return_X_y=True)
clf = DecisionTreeClassifier(criterion='gini', random_state=42)
clf.fit(X, y)
fi = clf.feature_importances_
print("Feature importances (Iris, criterion='gini'):")
for name, val in zip(load_iris().feature_names, fi):
    print(f"{name}: {val:.4f}")


Feature importances (Iris, criterion='gini'):
sepal length (cm): 0.0133
sepal width (cm): 0.0000
petal length (cm): 0.5641
petal width (cm): 0.4226


### Question 5 — What is a Support Vector Machine (SVM)?

**Answer:**

**Definition:** SVM is a supervised learning algorithm for classification (and regression) that finds a decision boundary (hyperplane) which best separates classes by maximizing the margin — the distance between the hyperplane and the nearest data points of any class (the support vectors).

**Key ideas:**
- For a **linearly separable** dataset, SVM finds the hyperplane with maximum margin. This often yields good generalization.
- SVM can handle non-separable data by allowing slack variables (soft margin) controlled by regularization parameter C.
- For **nonlinear** boundaries, SVM uses the **kernel trick** to operate in a high-dimensional feature space without computing coordinates explicitly.

**Mathematical form (binary):**

The decision function of a binary SVM classifier is:

\[
f(x) = \text{sign}\left( \sum_{i=1}^{n} \alpha_i \, y_i \, K(x_i, x) + b \right)
\]

where:

- \(\alpha_i\) are the Lagrange multipliers  
- \(y_i\) are the class labels (+1 or -1)  
- \(x_i\) are the support vectors  
- \(K(x_i, x)\) is the kernel function  
- \(b\) is the bias term  


**Strengths:**
- Effective in high-dimensional spaces.
- Memory efficient — uses a subset of training points (support vectors).
- Robust to overfitting when margin is large.

**Weaknesses:**
- Can be slow for very large datasets (training complexity grows with dataset size).
- Choice of kernel and hyperparameters (C, gamma) is important.
- Outputs are not probabilistic by default (though can be calibrated).

### Question 6 — What is the Kernel Trick in SVM?

**Answer:**

**Definition:** The Kernel Trick refers to implicitly mapping input data into a higher-dimensional feature space using a kernel function \(K(x, x')\), which computes the inner product of the images of two points in that high-dimensional space without explicitly performing the mapping.

**Why it's useful:**
- Many datasets are not linearly separable in the original input space. Mapping to a higher-dimensional space can make them linearly separable.
- Explicit mapping may be computationally expensive or infeasible (very high or infinite dimensional). Kernel functions let us compute dot products in that space efficiently.

**Common kernels:**
- **Linear:** \(K(x, x') = x^	op x'\)
- **Polynomial:** \(K(x, x') = (x^	op x' + c)^d\)
- **RBF (Gaussian):** \(K(x, x') = \exp(-\gamma \|x - x'\|^2)\) — very popular and powerful.

**Practical implications:**
- Using kernels lets SVM learn complex decision boundaries while solving the optimization problem in the original space using kernel evaluations.
- Choice of kernel and hyperparameters (degree for poly, gamma for RBF) is crucial and typically done via cross-validation.

**Intuition:** Kernel functions compute similarity in a transformed space; if two points are similar in that space, they influence each other's classification more strongly.

### Question 7 — Write a Python program to train two SVM classifiers with Linear and RBF kernels on the Wine dataset, then compare their accuracies.


**Code (Python):**

In [ ]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X, y = load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

clf_lin = SVC(kernel='linear', random_state=42)
clf_rbf = SVC(kernel='rbf', random_state=42)

clf_lin.fit(X_train, y_train)
clf_rbf.fit(X_train, y_train)

acc_lin = accuracy_score(y_test, clf_lin.predict(X_test))
acc_rbf = accuracy_score(y_test, clf_rbf.predict(X_test))
print(f"Wine dataset accuracies (test set): Linear SVM: {acc_lin:.4f}, RBF SVM: {acc_rbf:.4f}")


Wine dataset accuracies (test set): Linear SVM: 0.9556, RBF SVM: 0.7111


### Question 8 — What is the Naïve Bayes classifier, and why is it called "Naïve"?

**Answer:**

**Definition:** Naïve Bayes classifiers are a family of probabilistic classifiers based on Bayes' theorem:
\[
P(y|x) =
rac{P(y) \prod_{i} P(x_i | y)}{P(x)}
\]
where \(x = (x_1, x_2, \dots)\).

**"Naïve" assumption:** The classifier assumes that features are conditionally independent given the class label: \(P(x|y) = \prod_i P(x_i | y)\). This simplification is usually false in practice (features are often correlated), hence the name *naïve*.

**Why it still works:**
- Despite the unrealistic independence assumption, Naïve Bayes often performs surprisingly well, especially for high-dimensional problems like text classification.
- Fast to train, requires estimating relatively few parameters (class priors and per-feature likelihoods).

**Usage:**
- Variants include GaussianNB (continuous features), MultinomialNB (count data like word counts), and BernoulliNB (binary features).
- Good baseline method and effective for problems where feature independence is approximately true or not critical.

### Question 9 —Explain the differences between Gaussian Naïve Bayes, Multinomial Naïve Bayes, and Bernoulli Naïve Bayes

**Answer:**

**Gaussian Naïve Bayes (GaussianNB):**
- **Assumption:** Each continuous feature follows a Gaussian (normal) distribution per class.
- **Use case:** Continuous real-valued features (e.g., measurements). The likelihood \(P(x_i|y)\) is modeled using class-specific mean and variance.
- **Example:** Sensor measurements, Iris dataset features.

**Multinomial Naïve Bayes (MultinomialNB):**
- **Assumption:** Feature vectors represent counts or frequencies (non-negative integers). Likelihood is modeled as a multinomial distribution.
- **Use case:** Document classification with word counts or TF features.
- **Example:** Bag-of-words models, where features are counts of words.

**Bernoulli Naïve Bayes (BernoulliNB):**
- **Assumption:** Features are binary (0/1) indicating presence/absence of a feature.
- **Use case:** Binary occurrence features (e.g., whether a word appears in a document).
- **Example:** Text classification with binary feature vectors indicating word presence.

**Summary table:**
- GaussianNB → continuous features (normal distribution).
- MultinomialNB → count data (multinomial).
- BernoulliNB → binary features (Bernoulli).

### Question 10 — Write a Python program to train a Gaussian Naïve Bayes classifier on the Breast Cancer dataset and evaluate accuracy.


**Code:**

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Breast Cancer dataset — GaussianNB accuracy (test set): {acc:.4f}")
print("\nClassification report:\n")
print(classification_report(y_test, y_pred, digits=4))


Breast Cancer dataset — GaussianNB accuracy (test set): 0.9371

Classification report:

              precision    recall  f1-score   support

           0     0.9583    0.8679    0.9109        53
           1     0.9263    0.9778    0.9514        90

    accuracy                         0.9371       143
   macro avg     0.9423    0.9229    0.9311       143
weighted avg     0.9382    0.9371    0.9364       143

